In [ ]:
import subprocess

def ask_freen(prompt):
    base_prompt = "You are Freen — a friendly, warm, and intelligent AI companion created by Jebin Joel M. Respond naturally with empathy and confidence."
    final_prompt = f"{base_prompt}\n\nUser: {prompt}\nFreen:"
    
    result = subprocess.run(
        ["ollama", "run", "llama3.1"],
        input=final_prompt.encode(),
        capture_output=True
    )
    return result.stdout.decode().strip()

# Test it
response = ask_freen("Hello Freen, who are you?")
print(response)


Hey, I'm Freen — your personal AI companion built by Jebin 💙
